# Fictitious Play
This tutorial  demonstrates fictitious play on a zero-sum strategic (non-extensive) game. 

For the payoff matrix, row player is maximizer, coloumn player is minimizer.

Reference: https://towardsdatascience.com/introduction-to-fictitious-play-12a8bc4ed1bb

In [31]:
import numpy as np
! pip install pandas
import pandas as pd
# GameMatrix = np.array([[-2,3], [3,-4]])
GameMatrix = np.array([[0,2,-1], [-1,0,1], [1,-1,0]])

Itr = 10000
print('Game matrix: ')
pd.DataFrame(GameMatrix)


Game matrix: 


,0,1,2
0,0,2,-1
1,-1,0,1
2,1,-1,0


In [37]:
# a random initial step on row
row_value = np.zeros(GameMatrix.shape[0])
col_value = np.zeros(GameMatrix.shape[1])
min_id_list, max_id_list = [], []

for i in range(Itr):
    # current l is row
    min_id = np.argmin(row_value)
    min_id_list.append(min_id)
    l = GameMatrix[:, min_id] # l is column now
    col_value += np.array(l)
    
    # current l is column
    max_id = np.argmax(col_value)
    max_id_list.append(max_id)
    l = GameMatrix[max_id]  #  l is row now
    row_value += np.array(l)
    
# The statistical frequencies of occurrence of different entries are just their probability masses in final policy,
# which is the average best response over history.
hist, _ = np.histogram(max_id_list, bins=GameMatrix.shape[0])
max_policy = hist/np.sum(hist)
hist, _ = np.histogram(min_id_list, bins=GameMatrix.shape[1])
min_policy = hist/np.sum(hist)

row_value = row_value / (i+1)
col_value = col_value / (i+1)
print(f'For row player, strategy is {max_policy}, game value [lower bound, upper bound]: {row_value}')
print(f'For column player, strategy is {min_policy}, game value [lower bound, upper bound]: {col_value}')


For row player, strategy is [0.25   0.3332 0.4168], game value [lower bound, upper bound]: [0.0836 0.0832 0.0832]
For column player, strategy is [0.3333 0.2499 0.4168], game value [lower bound, upper bound]: [0.083  0.0835 0.0834]


In [39]:
a=[1]
b,c=a

ValueError: not enough values to unpack (expected 2, got 1)